In [1]:
import json
import requests
import time
import os
import pandas as pd

### making data a bit easier to see
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

### should be outside the repo
cred_location = r'/Users/jj/code/strava_creds/'
data_location = r'/Users/jj/code/strava_data/'

## Authorisation
This code will make sure you have the necessary tokens to authenticate to the strava API

full documentation for this can be found here https://developers.strava.com/

In [2]:
### You need to have created an app on strava - this is much easier than it sounds.
### Go through the first couple of steps here if unsure: 
###https://medium.com/@annthurium/getting-started-with-the-strava-api-a-tutorial-f3909496cd2d

### These client tokens don't change, so you should only have to enter these once. 
def get_client_tokens():
    print('please go here https://www.strava.com/settings/api, and copy the token information as prompted')
    client_id = input('please enter your client id')
    client_secret = input('please enter your client secret')
    redirect_uri = input('please enter a redirect uri (use http://localhost/ if unsure)')

    client_tokens = {
        'client_id':str(client_id),
        'client_secret':str(client_secret),
        'redirect_uri':str(redirect_uri)
    }

    with open(cred_location+'client_tokens.json', 'w') as outfile:
        json.dump(client_tokens, outfile, indent=4)

    print("client tokens saved")
    return None

### This will just check if the client tokens exist, not if they are valid.
### client token, client secret, and refresh token should all be held in a json file called client_tokens.json
def check_client_tokens():
    try:
        client_tokens = json.load(open(cred_location+'client_tokens.json'))
        if {'client_id', 'client_secret', 'redirect_uri'} == set(client_tokens.keys()):
            print('Client Tokens Found')
            return True
        else:
            return False
    except:
        return False


def prompt_authorization():

      client_tokens = json.load(open(cred_location+'client_tokens.json'))

      ### necessary to get athlete activities
      scopes = ','.join(['profile:read_all', 'activity:read_all'])

      # Authorization URL
      request_url = f'http://www.strava.com/oauth/authorize?client_id={client_tokens["client_id"]}' \
                        f'&response_type=code&redirect_uri={client_tokens["redirect_uri"]}' \
                        f'&approval_prompt=force' \
                        f'&scope={scopes}'

      # This will prompt you and ask for the code in the url
      print('Click here:', request_url)
      print('Please authorize the app and copy&paste below the generated code!')
      print('P.S: you can find the code in the URL between "code=" and the nexrt "&"')
      code = input('Insert the code from the url: ')

      # Get the access token
      token = requests.post(url='https://www.strava.com/api/v3/oauth/token',
                        data={'client_id': client_tokens["client_id"],
                              'client_secret': client_tokens["client_secret"],
                              'code': code,
                              'grant_type': 'authorization_code'})

      access_tokens = token.json()

      # save the token with the applied for scope in the cred_location
      access_tokens.update({'scopes':scopes})

      with open(cred_location+'access_tokens.json', 'w') as outfile:
            json.dump(access_tokens, outfile, indent=4)
      print('Authorisation Complete')



### access tokens expire every 6 hours.
### the access token can be refreshed using this code.
### it will have the same scope as the original access code.
def refresh_authorization():

      # get previously saved tokens
      client_tokens = json.load(open(cred_location+'client_tokens.json'))
      access_tokens = json.load(open(cred_location+'access_tokens.json'))

      refresh_url = "https://www.strava.com/oauth/token"

      payload = {
            'client_secret':client_tokens['client_secret'],
            'client_id':client_tokens['client_id'],
            'refresh_token':access_tokens['refresh_token'],
            'grant_type':'refresh_token',
            'f':'json'
                 }
      
      print("Requesting Token...\n")
      res = requests.post(refresh_url, data=payload, verify=False)
      access_token = res.json()

      # save the new token and copy the scope of the old token
      access_token.update({'scopes':access_tokens['scopes']})

      if res.status_code < 300:
            with open(cred_location+'access_tokens.json', 'w') as outfile:
                  json.dump(access_token, outfile, indent=4)
            print('Authorisation Complete')

      else:
            print('Authorisation Error')
            print(res.json())

      return True





def authorization_flow():

      ### Check if we have client tokens
      if check_client_tokens():
            
            ### check we have an access token
            if os.path.isfile(cred_location+'access_tokens.json'):
                  tokens = json.load(open(cred_location+'access_tokens.json'))
                  
                  ### check if we have the right keys in the access token file
                  if all(x in tokens.keys() for x in ['access_token', 'scopes', 'refresh_token']):

                        ### check if the access token is still valid
                        if time.time() < int(tokens['expires_at']):
                              print((int(tokens['expires_at']) - int(time.time())) / 60, 'minutes until token expires')
                              print('current scopes:', tokens['scopes'])
                        
                        else:
                              print('access_code_expired, requesting refresh')
                              refresh_authorization()
                              authorization_flow() ## restart the flow

                  else:
                        print('data missing from access_tokens.json')
                        prompt_authorization()
                        authorization_flow() ## restart the flow

            else: 
                  print('access_token.json file missing')
                  prompt_authorization()
                  authorization_flow()

      else:
           print('client tokens not found')
           get_client_tokens()
           authorization_flow()

#prompt_authorization()                 
authorization_flow()


Client Tokens Found
356.8666666666667 minutes until token expires
current scopes: profile:read_all,activity:read_all


## Get Activity Data
this code will hit the 'all_activities' endpoint to get all data for the authenticated user

more here https://developers.strava.com/docs/reference/#api-Activities-getLoggedInAthleteActivities

In [3]:
### Makes one request to the API for n number of activities
def get_activities(n, page_no=1):
    activites_url = "https://www.strava.com/api/v3/athlete/activities"

    access_tokens = json.load(open(cred_location+'access_tokens.json'))

    header = {'Authorization': 'Bearer ' + access_tokens['access_token']}
    param = {'per_page': n, 'page': page_no}

    response = requests.get(activites_url, headers=header, params=param)
    return response

### preview of what they look like
two_activities = get_activities(2)
trial_df = pd.DataFrame(two_activities.json())
trial_df


,resource_state,athlete,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,id,start_date,start_date_local,timezone,utc_offset,location_city,location_state,location_country,achievement_count,kudos_count,comment_count,athlete_count,photo_count,map,trainer,commute,manual,private,visibility,flagged,gear_id,start_latlng,end_latlng,average_speed,max_speed,average_temp,average_watts,kilojoules,device_watts,has_heartrate,average_heartrate,max_heartrate,heartrate_opt_out,display_hide_heartrate_option,elev_high,elev_low,upload_id,upload_id_str,external_id,from_accepted_tag,pr_count,total_photo_count,has_kudoed,suffer_score
0,2,"{'id': 23883723, 'resource_state': 1}",Lunch Ride,2037.2,278,775,0.0,Ride,Ride,None,8726797885,2023-03-16T22:49:16Z,2023-03-17T11:49:16Z,(GMT+12:00) Pacific/Auckland,46800.0,None,None,,0,0,0,1,0,"{'id': 'a8726797885', 'summary_polyline': 'zj_...",False,False,False,True,only_me,False,b4083088,"[-41.28829252906144, 174.7669970523566]","[-41.2935836892575, 174.77822947315872]",7.328,15.816,27,26.4,7.3,False,True,113.5,136.0,False,True,111.6,-14.0,9366533974,9366533974,garmin_ping_264691976304,False,0,0,False,0.0
1,2,"{'id': 23883723, 'resource_state': 1}",Lunch Ride,2167.2,727,773,132.0,Ride,Ride,None,8721532095,2023-03-15T23:29:03Z,2023-03-16T12:29:03Z,(GMT+12:00) Pacific/Auckland,46800.0,None,None,,0,0,0,1,0,"{'id': 'a8721532095', 'summary_polyline': '`a`...",False,False,False,True,only_me,False,b4083088,"[-41.293124444782734, 174.77520930580795]","[-41.28730338066816, 174.76581109687686]",2.981,5.510,28,146.0,106.1,False,True,125.4,145.0,False,True,134.4,32.6,9361000061,9361000061,garmin_ping_264550698410,False,0,0,False,3.0


In [4]:
### this iterates through the get_activities() function to get all activities for an athlete
### You should check that the total number of activities matches what's on your strava profile
def get_all_activities():

    output = []
    i = 1
    valid_data = True
    while valid_data == True:
        response = get_activities(200, i)
        if response.status_code < 300 and len(response.json()) > 0: 
            [output.append(x) for x in response.json()]
            print(len(response.json()), 'rows added from page', i)
            i+=1
        else:
            print(response.json())
            valid_data = False

    return output

all_activities = get_all_activities()
all_activities_df = pd.DataFrame(all_activities)

### save outside the repo
all_activities_df.to_csv(data_location+'all_activities_raw.csv')
all_activities_df.head()

200 rows added from page 1
200 rows added from page 2
200 rows added from page 3
200 rows added from page 4
200 rows added from page 5
200 rows added from page 6
200 rows added from page 7
200 rows added from page 8
200 rows added from page 9
200 rows added from page 10
200 rows added from page 11
6 rows added from page 12
[]


,resource_state,athlete,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,id,start_date,start_date_local,timezone,utc_offset,location_city,location_state,location_country,achievement_count,kudos_count,comment_count,athlete_count,photo_count,map,trainer,commute,manual,private,visibility,flagged,gear_id,start_latlng,end_latlng,average_speed,max_speed,average_temp,average_watts,kilojoules,device_watts,has_heartrate,average_heartrate,max_heartrate,heartrate_opt_out,display_hide_heartrate_option,elev_high,elev_low,upload_id,upload_id_str,external_id,from_accepted_tag,pr_count,total_photo_count,has_kudoed,suffer_score,average_cadence,max_watts,weighted_average_watts
0,2,"{'id': 23883723, 'resource_state': 1}",Lunch Ride,2037.2,278,775,0.0,Ride,Ride,NaN,8726797885,2023-03-16T22:49:16Z,2023-03-17T11:49:16Z,(GMT+12:00) Pacific/Auckland,46800.0,None,None,,0,0,0,1,0,"{'id': 'a8726797885', 'summary_polyline': 'zj_...",False,False,False,True,only_me,False,b4083088,"[-41.28829252906144, 174.7669970523566]","[-41.2935836892575, 174.77822947315872]",7.328,15.816,27.0,26.4,7.3,False,True,113.5,136.0,False,True,111.6,-14.0,9.366534e+09,9366533974,garmin_ping_264691976304,False,0,0,False,0.0,NaN,NaN,NaN
1,2,"{'id': 23883723, 'resource_state': 1}",Lunch Ride,2167.2,727,773,132.0,Ride,Ride,NaN,8721532095,2023-03-15T23:29:03Z,2023-03-16T12:29:03Z,(GMT+12:00) Pacific/Auckland,46800.0,None,None,,0,0,0,1,0,"{'id': 'a8721532095', 'summary_polyline': '`a`...",False,False,False,True,only_me,False,b4083088,"[-41.293124444782734, 174.77520930580795]","[-41.28730338066816, 174.76581109687686]",2.981,5.510,28.0,146.0,106.1,False,True,125.4,145.0,False,True,134.4,32.6,9.361000e+09,9361000061,garmin_ping_264550698410,False,0,0,False,3.0,NaN,NaN,NaN
2,2,"{'id': 23883723, 'resource_state': 1}",Lunch Ride,2109.1,267,302,3.0,Ride,Ride,NaN,8721256990,2023-03-15T22:56:29Z,2023-03-16T11:56:29Z,(GMT+12:00) Pacific/Auckland,46800.0,None,None,,0,0,0,1,0,"{'id': 'a8721256990', 'summary_polyline': 'nj_...",False,False,False,True,only_me,False,b4083088,"[-41.28733104094863, 174.76634301245213]","[-41.29523408599198, 174.77632116526365]",7.899,14.708,28.0,33.5,8.9,False,True,105.8,121.0,False,True,114.0,2.0,9.360712e+09,9360712096,garmin_ping_264543599077,False,0,0,False,0.0,NaN,NaN,NaN
3,2,"{'id': 23883723, 'resource_state': 1}",Afternoon Mountain Bike Ride,33168.0,11823,20633,1089.9,Ride,MountainBikeRide,10.0,8716585507,2023-03-15T00:14:37Z,2023-03-15T13:14:37Z,(GMT+12:00) Pacific/Auckland,46800.0,None,None,,19,5,0,1,0,"{'id': 'a8716585507', 'summary_polyline': 'v{_...",False,False,False,False,everyone,False,b11952969,"[-41.292270831763744, 174.77877404540777]","[-41.287265829741955, 174.76577061228454]",2.805,12.458,28.0,140.8,1664.1,False,True,144.5,183.0,False,True,183.1,7.7,9.355787e+09,9355786561,garmin_ping_264429216355,False,8,0,False,159.0,NaN,NaN,NaN
4,2,"{'id': 23883723, 'resource_state': 1}",Morning Mountain Bike Ride,2300.3,449,525,2.0,Ride,MountainBikeRide,NaN,8715133554,2023-03-14T20:49:41Z,2023-03-15T09:49:41Z,(GMT+12:00) Pacific/Auckland,46800.0,None,None,,0,0,0,1,0,"{'id': 'a8715133554', 'summary_polyline': 'rj_...",False,False,False,True,only_me,False,b4083088,"[-41.287781400606036, 174.76704055443406]","[-41.291415710002184, 174.77939656935632]",5.123,13.491,24.0,40.2,18.1,False,True,106.6,144.0,False,True,267.8,150.2,9.354259e+09,9354258948,garmin_ping_264381823955,False,0,0,False,1.0,NaN,NaN,NaN


## Test Activity Data
A few tests to check the data from the API is as expected

In [6]:
### Test 1: Check that all your activities have been collected

### before doing this, go to https://www.strava.com/dashboard and make a note of your total number of activities

### check that the dataframe is of the same length.
print(len(all_activities_df))

### test passed if the numbers match

2206


In [7]:
### Test 2: check that the columns are populated
columns_that_must_be_populated = ['id', 'name', 'type', 'start_date', 'private']
possible_empty_values = ['none', 'unknown', 'None', 'Unknown', '', None]

errors = []
for v in possible_empty_values:
    for c in columns_that_must_be_populated:
        if v in all_activities_df[c]:
            errors.append({c:(df[c] == v).sum()+' instances of '+v})
if errors == []:
    print('TEST PASSED')
else:
    print('TEST FAILED')
    print(errors)

TEST PASSED


In [8]:
### Test 3: Check that columns which should be unique are unique
columns_that_must_be_unique = ['id', 'start_date']
if all([all_activities_df[c].is_unique for c in columns_that_must_be_unique]):
    print('TEST PASSED')
else:
    print('TEST_FAILED')
    print([c for c in columns_that_must_be_unique if df[c].is_unique is False])

TEST PASSED


## Get User Data

This code will hit the 'Athlete' endpoint.

https://developers.strava.com/docs/reference/#api-Athletes-getLoggedInAthlete

In [9]:
def get_athlete_data():
    athlete_url = "https://www.strava.com/api/v3/athlete"
    access_tokens = json.load(open(cred_location+'access_tokens.json'))

    header = {'authorization': 'Bearer ' + access_tokens['access_token']}

    response = requests.get(athlete_url, headers=header)

    return response.json()

athlete_data = get_athlete_data()
with open(data_location+'athlete_data_raw.json', 'w') as outfile:
    json.dump(athlete_data, outfile, indent=4)

athlete_data

{'id': 23883723,
 'username': None,
 'resource_state': 3,
 'firstname': 'Jonathan',
 'lastname': 'Jackson',
 'bio': '',
 'city': 'Wellington',
 'state': '',
 'country': '',
 'sex': 'M',
 'premium': True,
 'summit': True,
 'created_at': '2017-08-02T10:35:23Z',
 'updated_at': '2023-01-18T20:47:55Z',
 'badge_type_id': 1,
 'weight': 82.3,
 'profile_medium': 'https://graph.facebook.com/10209743672604594/picture?height=256&width=256',
 'profile': 'https://graph.facebook.com/10209743672604594/picture?height=256&width=256',
 'friend': None,
 'follower': None,
 'blocked': False,
 'can_follow': True,
 'follower_count': 55,
 'friend_count': 67,
 'mutual_friend_count': 0,
 'athlete_type': 0,
 'date_preference': '%m/%d/%Y',
 'measurement_preference': 'meters',
 'clubs': [{'id': 253800,
   'resource_state': 2,
   'name': 'VUW Road Cycling Club',
   'profile_medium': 'https://dgalywyr863hv.cloudfront.net/pictures/clubs/253800/6203384/1/medium.jpg',
   'profile': 'https://dgalywyr863hv.cloudfront.net/

## Test User Data

In [29]:
### Test 1: Check that some fields we expect to be populated are populated:
fields_which_should_be_populated = ['firstname', 'id', 'created_at', 'follower_count']

if all([x for x in fields_which_should_be_populated if x in athlete_data.keys() and athlete_data[x] == True]):
    print('TEST PASSED')
else:
    print('TEST FAILED')
    print('values missing: ', [x for x in fields_which_should_be_populated if x not in athlete_data.keys() or athlete_data[x] == False])

TEST PASSED


In [30]:
### Test 2: Check that athlete gear matches activity gear
activity_gear = set(all_activities_df['gear_id'].dropna().unique())
athlete_gear = {b['id'] for b in athlete_data['bikes']}

if athlete_gear == activity_gear:
    print('TEST PASSED')
else:
    print('TEST FAILED')

TEST PASSED
